In [14]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from collections import Counter
import json
import re
from sklearn.ensemble import RandomForestClassifier
import networkx as nx

import seaborn as sns
sns.set_style('darkgrid')

In [15]:
df_1 = pd.read_csv("df_1.csv", header = 0)
df_0 = pd.read_csv("df_0.csv", header = 0)
df_public = pd.read_csv("df_public.csv", header = 0)
# print(df_1)
# print(df_0)
# print(df_public)

In [16]:
# generate df_node

df_node = pd.DataFrame(columns = ['id', 'NeighbourList', 'EdgeNum', 'KeyList', 'VenueList', 'PaperNum', 'FirstYear', 'LastYear'])

# read json

f_json = open("nodes.json")
node_Info = f_json.read()
authors = json.loads(node_Info)

f_json.close()

for author in authors:
    id = author[str('id')]
    first = author[str('first')]
    last = author[str('last')]
    paper = author[str('num_papers')]
    key = []
    venue = []
    
    for k_num in range(53):
        keyword = 'keyword_' + str(k_num)
        if keyword in author.keys():
            key.append(k_num)
    for v_num in range(348):
        vname = 'venue_' + str(v_num)
        if vname in author.keys():
            venue.append(v_num)
    
    insertRow = pd.DataFrame([[id, 0, 0, key, venue, paper, first, last]],
                             columns = ['id', 'NeighbourList', 'EdgeNum', 'KeyList', 'VenueList', 'PaperNum', 'FirstYear', 'LastYear'])
    
    df_node = df_node.append(insertRow, ignore_index = True)
    
print(df_node)

        id NeighbourList EdgeNum  \
0        0             0       0   
1        1             0       0   
2        2             0       0   
3        3             0       0   
4        4             0       0   
...    ...           ...     ...   
4080  4080             0       0   
4081  4081             0       0   
4082  4082             0       0   
4083  4083             0       0   
4084  4084             0       0   

                                                KeyList  \
0     [0, 2, 6, 8, 10, 13, 15, 33, 34, 37, 38, 41, 4...   
1     [0, 1, 2, 4, 6, 7, 8, 10, 11, 13, 15, 17, 18, ...   
2             [0, 2, 3, 13, 18, 25, 31, 37, 40, 41, 43]   
3     [0, 2, 4, 6, 10, 11, 15, 18, 21, 23, 25, 26, 2...   
4     [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...   
...                                                 ...   
4080           [0, 4, 6, 7, 13, 15, 31, 37, 46, 48, 50]   
4081  [0, 2, 4, 6, 7, 9, 10, 15, 26, 31, 33, 34, 37,...   
4082              [0, 2, 4, 10, 22, 

In [17]:
# insert NeighbourList and EdgeNum into df_node

data_source = []
data_sink = []
N_neighbour = 0
df_test = pd.DataFrame(columns = ['NeighbourList', 'EdgeNum'])

f_txt = open("train.txt", 'r')

for line in f_txt:
    x = line.split(' ')
    y = x[1].strip('\n').split('\t')
    data_source = int(x[0])
    data_sink = []
    
    for i in y:
        data_sink.append(int(i))
        
    N_neighbour = len(data_sink)
    df_node.loc[data_source, 'NeighbourList'] = data_sink
    df_node.loc[data_source, 'EdgeNum'] = N_neighbour  


In [18]:
print(df_node)

        id                                      NeighbourList EdgeNum  \
0        0                                       [2860, 3117]       2   
1        1  [318, 1610, 1776, 1865, 2283, 2507, 3076, 3108...      12   
2        2                 [164, 413, 1448, 1650, 3119, 3238]       6   
3        3  [164, 172, 712, 849, 872, 983, 1616, 1826, 197...      15   
4        4  [95, 97, 143, 196, 317, 374, 375, 482, 526, 54...      74   
...    ...                                                ...     ...   
4080  4080                                        [910, 1300]       2   
4081  4081             [137, 162, 384, 860, 1278, 3656, 3819]       7   
4082  4082                [642, 2291, 2310, 2350, 2993, 3097]       6   
4083  4083                                 [1108, 3423, 3698]       3   
4084  4084     [170, 481, 1065, 1348, 2101, 2321, 3244, 3879]       8   

                                                KeyList  \
0     [0, 2, 6, 8, 10, 13, 15, 33, 34, 37, 38, 41, 4...   
1    

In [19]:
# use df_node to update df_public

df_public['CommonNeighbour'] = np.nan    #nan:not a number
df_public['PopularDeg'] = np.nan
df_public['CommonKey'] = np.nan
df_public['CommonVenue'] = np.nan
df_public['PaperNumDiff'] = np.nan
df_public['FirstYear'] = np.nan
df_public['LastYear'] = np.nan
df_public['CommonPeriod'] = np.nan
df_public['KeyPlus'] = np.nan
df_public['VenuePlus'] = np.nan
df_public['PaperNumPlus'] = np.nan



for index, row in df_public.iterrows():
    
    source = int(row['Source'])
    sink = int(row['Sink'])
    
    if not df_node.loc[source, 'NeighbourList'] or not df_node.loc[sink, 'NeighbourList']:
        commonneighbour = 0
    else:
        source_neighbour = set(df_node.loc[source, 'NeighbourList'])
        sink_neighbour = set(df_node.loc[sink, 'NeighbourList'])
        commonneighbour = len(source_neighbour.intersection(sink_neighbour))

    source_popular = int(df_node.loc[source, 'EdgeNum'])
    sink_popular = int(df_node.loc[sink, 'EdgeNum'])
    populardeg = source_popular * sink_popular
    
    source_key = set(df_node.loc[source, 'KeyList'])
    sink_key = set(df_node.loc[sink, 'KeyList'])
    commonkey = len(source_key.intersection(sink_key))
    keyplus = len(source_key) * len(sink_key)
    
    source_venue = set(df_node.loc[source, 'VenueList'])
    sink_venue = set(df_node.loc[sink, 'VenueList'])
    commonvenue = len(source_venue.intersection(sink_venue))
    venueplus = len(source_venue) * len(sink_venue)
    
    source_paper = int(df_node.loc[source, 'PaperNum'])
    sink_paper = int(df_node.loc[sink, 'PaperNum'])
    papernumdiff = abs(source_paper - sink_paper)
    papernumplus = source_paper * sink_paper
    
    source_first = int(df_node.loc[source, 'FirstYear'])
    sink_first = int(df_node.loc[sink, 'FirstYear'])
    first = abs(source_first - sink_first)
    
    source_last = int(df_node.loc[source, 'LastYear'])
    sink_last = int(df_node.loc[sink, 'LastYear'])
    last = abs(source_last - sink_last)
    
    overlap = min(source_first, sink_first) - max(sink_last,source_last)
    
    df_public.loc[index, 'CommonNeighbour'] = commonneighbour
    df_public.loc[index, 'PopularDeg'] = populardeg
    df_public.loc[index, 'CommonKey'] = commonkey
    df_public.loc[index, 'CommonVenue'] = commonvenue
    df_public.loc[index, 'PaperNumDiff'] = papernumdiff
    df_public.loc[index, 'FirstYear'] = first
    df_public.loc[index, 'LastYear'] = last
    df_public.loc[index, 'CommonPeriod'] = overlap
    df_public.loc[index, 'KeyPlus'] = keyplus
    df_public.loc[index, 'VenuePlus'] = venueplus
    df_public.loc[index, 'PaperNumPlus'] = papernumplus
    
print(df_public)
    

      Source  Sink  CommonNeighbour  PopularDeg  CommonKey  CommonVenue  \
0          1  4021              1.0        36.0       14.0          1.0   
1          2  3795              3.0       306.0       11.0          0.0   
2          3  1627              0.0        75.0        4.0          3.0   
3          4  2206              2.0       222.0       17.0          4.0   
4          4  2575              1.0       222.0       13.0          2.0   
...      ...   ...              ...         ...        ...          ...   
1995    3864  3970              0.0       602.0        9.0          2.0   
1996    3884  3921              0.0       225.0       10.0          3.0   
1997    3946  3956              0.0        70.0       12.0          0.0   
1998    3956  3969              1.0        35.0       13.0          1.0   
1999    4024  4059              0.0       984.0       18.0          3.0   

      PaperNumDiff  FirstYear  LastYear  CommonPeriod  KeyPlus  VenuePlus  \
0              8.0    

In [20]:
# use df_node to update df_1

df_1['CommonNeighbour'] = np.nan    #nan:not a number
df_1['PopularDeg'] = np.nan
df_1['CommonKey'] = np.nan
df_1['CommonVenue'] = np.nan
df_1['PaperNumDiff'] = np.nan
df_1['FirstYear'] = np.nan
df_1['LastYear'] = np.nan
df_1['CommonPeriod'] = np.nan
df_1['KeyPlus'] = np.nan
df_1['VenuePlus'] = np.nan
df_1['PaperNumPlus'] = np.nan
df_1['Label'] = np.nan



for index, row in df_1.iterrows():
    
    source = int(row['source'])
    sink = int(row['sink'])
    
    if not df_node.loc[source, 'NeighbourList'] or not df_node.loc[sink, 'NeighbourList']:
        commonneighbour = 0
    else:
        source_neighbour = set(df_node.loc[source, 'NeighbourList'])
        sink_neighbour = set(df_node.loc[sink, 'NeighbourList'])
        commonneighbour = len(source_neighbour.intersection(sink_neighbour))

    source_popular = int(df_node.loc[source, 'EdgeNum'])
    sink_popular = int(df_node.loc[sink, 'EdgeNum'])
    populardeg = source_popular * sink_popular
    
    source_key = set(df_node.loc[source, 'KeyList'])
    sink_key = set(df_node.loc[sink, 'KeyList'])
    commonkey = len(source_key.intersection(sink_key))
    keyplus = len(source_key) * len(sink_key)
    
    source_venue = set(df_node.loc[source, 'VenueList'])
    sink_venue = set(df_node.loc[sink, 'VenueList'])
    commonvenue = len(source_venue.intersection(sink_venue))
    venueplus = len(source_venue) * len(sink_venue)
    
    source_paper = int(df_node.loc[source, 'PaperNum'])
    sink_paper = int(df_node.loc[sink, 'PaperNum'])
    papernumdiff = abs(source_paper - sink_paper)
    papernumplus = source_paper * sink_paper
    
    source_first = int(df_node.loc[source, 'FirstYear'])
    sink_first = int(df_node.loc[sink, 'FirstYear'])
    first = abs(source_first - sink_first)
    
    source_last = int(df_node.loc[source, 'LastYear'])
    sink_last = int(df_node.loc[sink, 'LastYear'])
    last = abs(source_last - sink_last)
    
    overlap = min(source_first, sink_first) - max(sink_last,source_last)
    
    df_1.loc[index, 'CommonNeighbour'] = commonneighbour
    df_1.loc[index, 'PopularDeg'] = populardeg
    df_1.loc[index, 'CommonKey'] = commonkey
    df_1.loc[index, 'CommonVenue'] = commonvenue
    df_1.loc[index, 'PaperNumDiff'] = papernumdiff
    df_1.loc[index, 'FirstYear'] = first
    df_1.loc[index, 'LastYear'] = last
    df_1.loc[index, 'CommonPeriod'] = overlap
    df_1.loc[index, 'KeyPlus'] = keyplus
    df_1.loc[index, 'VenuePlus'] = venueplus
    df_1.loc[index, 'PaperNumPlus'] = papernumplus
    df_1.loc[index, 'Label'] = 1
    
print(df_1)
    
    

       source  sink  CommonNeighbour  PopularDeg  CommonKey  CommonVenue  \
0           0  2860              1.0         4.0       14.0          1.0   
1           0  3117              1.0         6.0       11.0          2.0   
2           1   318              4.0        72.0       17.0          3.0   
3           1  1610              0.0        96.0       18.0          2.0   
4           1  1776              1.0        72.0       12.0          3.0   
...       ...   ...              ...         ...        ...          ...   
53867    4084  1348              4.0       296.0       14.0          4.0   
53868    4084  2101              4.0        56.0       12.0          3.0   
53869    4084  2321              4.0        48.0       12.0          4.0   
53870    4084  3244              7.0       184.0       13.0          3.0   
53871    4084  3879              4.0       232.0       12.0          2.0   

       PaperNumDiff  FirstYear  LastYear  CommonPeriod  KeyPlus  VenuePlus  \
0        

In [21]:
# use df_node to update df_0

df_0['CommonNeighbour'] = np.nan    #nan:not a number
df_0['PopularDeg'] = np.nan
df_0['CommonKey'] = np.nan
df_0['CommonVenue'] = np.nan
df_0['PaperNumDiff'] = np.nan
df_0['FirstYear'] = np.nan
df_0['LastYear'] = np.nan
df_0['CommonPeriod'] = np.nan
df_0['KeyPlus'] = np.nan
df_0['VenuePlus'] = np.nan
df_0['PaperNumPlus'] = np.nan
df_0['Label'] = np.nan



for index, row in df_0.iterrows():
    
    source = int(row['source'])
    sink = int(row['sink'])
    
    if not df_node.loc[source, 'NeighbourList'] or not df_node.loc[sink, 'NeighbourList']:
        commonneighbour = 0
    else:
        source_neighbour = set(df_node.loc[source, 'NeighbourList'])
        sink_neighbour = set(df_node.loc[sink, 'NeighbourList'])
        commonneighbour = len(source_neighbour.intersection(sink_neighbour))

    source_popular = int(df_node.loc[source, 'EdgeNum'])
    sink_popular = int(df_node.loc[sink, 'EdgeNum'])
    populardeg = source_popular * sink_popular
    
    source_key = set(df_node.loc[source, 'KeyList'])
    sink_key = set(df_node.loc[sink, 'KeyList'])
    commonkey = len(source_key.intersection(sink_key))
    keyplus = len(source_key) * len(sink_key)
    
    source_venue = set(df_node.loc[source, 'VenueList'])
    sink_venue = set(df_node.loc[sink, 'VenueList'])
    commonvenue = len(source_venue.intersection(sink_venue))
    venueplus = len(source_venue) * len(sink_venue)
    
    source_paper = int(df_node.loc[source, 'PaperNum'])
    sink_paper = int(df_node.loc[sink, 'PaperNum'])
    papernumdiff = abs(source_paper - sink_paper)
    papernumplus = source_paper * sink_paper
    
    source_first = int(df_node.loc[source, 'FirstYear'])
    sink_first = int(df_node.loc[sink, 'FirstYear'])
    first = abs(source_first - sink_first)
    
    source_last = int(df_node.loc[source, 'LastYear'])
    sink_last = int(df_node.loc[sink, 'LastYear'])
    last = abs(source_last - sink_last)
    
    overlap = min(source_first, sink_first) - max(sink_last,source_last)
    
    df_0.loc[index, 'CommonNeighbour'] = commonneighbour
    df_0.loc[index, 'PopularDeg'] = populardeg
    df_0.loc[index, 'CommonKey'] = commonkey
    df_0.loc[index, 'CommonVenue'] = commonvenue
    df_0.loc[index, 'PaperNumDiff'] = papernumdiff
    df_0.loc[index, 'FirstYear'] = first
    df_0.loc[index, 'LastYear'] = last
    df_0.loc[index, 'CommonPeriod'] = overlap
    df_0.loc[index, 'KeyPlus'] = keyplus
    df_0.loc[index, 'VenuePlus'] = venueplus
    df_0.loc[index, 'PaperNumPlus'] = papernumplus
    df_0.loc[index, 'Label'] = 0
    
print(df_0)

       source  sink  CommonNeighbour  PopularDeg  CommonKey  CommonVenue  \
0         422   717              0.0        56.0        3.0          0.0   
1        3928  3376              0.0        18.0       15.0          1.0   
2         413  3895              0.0       100.0        7.0          0.0   
3        3017  1096              0.0        55.0       17.0          3.0   
4        3146   798              0.0        44.0       10.0          1.0   
...       ...   ...              ...         ...        ...          ...   
53867     489  3642              0.0        72.0       15.0          1.0   
53868    2364  2398              0.0        48.0        7.0          1.0   
53869    2875  1522              0.0         5.0       10.0          0.0   
53870     218  3356              0.0        50.0        7.0          1.0   
53871    1617  3391              0.0        44.0        6.0          0.0   

       PaperNumDiff  FirstYear  LastYear  CommonPeriod  KeyPlus  VenuePlus  \
0        

In [22]:
df_pos_dist = pd.read_csv("positive_0.csv", header = 0)
df_neg_dist = pd.read_csv("negative_0.csv", header = 0)
df_public_dist = pd.read_csv("test_0.csv", header = 0)
# print(df_pos_dist, df_neg_dist, df_public_dist)

df_1['distance'] = df_pos_dist
df_0['distance'] = df_neg_dist
df_public['distance'] = df_public_dist

# print(df_1,df_0,df_public)



In [23]:
all_data = df_1.append(df_0)

# data_01 = all_data.to_csv('data_01.csv')

y = all_data[['Label']]
X = all_data.drop(columns=['Label','source','sink','FirstYear','LastYear']) 
#print(all_data)

public = df_public.drop(columns=['Source','Sink','FirstYear','LastYear']) 

print(y)
print(X)
print(public)

       Label
0        1.0
1        1.0
2        1.0
3        1.0
4        1.0
...      ...
53867    0.0
53868    0.0
53869    0.0
53870    0.0
53871    0.0

[107744 rows x 1 columns]
       CommonNeighbour  PopularDeg  CommonKey  CommonVenue  PaperNumDiff  \
0                  1.0         4.0       14.0          1.0           1.0   
1                  1.0         6.0       11.0          2.0           2.0   
2                  4.0        72.0       17.0          3.0           9.0   
3                  0.0        96.0       18.0          2.0           7.0   
4                  1.0        72.0       12.0          3.0          11.0   
...                ...         ...        ...          ...           ...   
53867              0.0        72.0       15.0          1.0          18.0   
53868              0.0        48.0        7.0          1.0           2.0   
53869              0.0         5.0       10.0          0.0           2.0   
53870              0.0        50.0        7.0          1.

In [48]:
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import preprocessing

#all_data.groupby('Label').mean()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.01, random_state=1991)


# clf = RandomForestClassifier(n_estimators = 100)                              #RandomForest 
# clf = LogisticRegression()                                                    #LogisticRegression
clf = MLPClassifier(activation='logistic', solver='adam', alpha=0.00004)        #neural_network
clf.fit(X_train, y_train)
y_pred = clf.predict_proba(X_test)
# print(y_pred)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:921: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [49]:
y_public_RandomForest = clf.predict_proba(public)
#print(y_public_RandomForest)
print(y_public_RandomForest.shape)

output = pd.DataFrame(y_public_RandomForest)
output.to_csv('18_nn_4.csv')
# print(output)

(2000, 2)


In [50]:
prediction = clf.predict(X_test)
fpr,tpr,thresholds = metrics.roc_curve(y_test,prediction)

print(metrics.auc(fpr,tpr))

0.9753207218960644
